<a href="https://colab.research.google.com/github/ekshustova/ekshustova/blob/main/Recsys_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Callable, List

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as scs
from numpy.linalg import norm

In [112]:
# Скачиваем архив с данными MovieLens 1M
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -O ml-1m.zip

# Распаковываем архив в папку data
!unzip -uo ml-1m.zip -d data

# Приводим файл с рейтингами к формату CSV
!awk 'BEGIN { print "user,item,values,timestamp" } { gsub("::", ",", $0); print $0 }' data/ml-1m/ratings.dat > data/ml-1m/ratings.csv

# Приводим файл с фильмами к формату CSV
!awk 'BEGIN { print "item,title,genres" } { gsub("::", ",", $0); print $0 }' data/ml-1m/movies.dat > data/ml-1m/movies.csv


--2025-01-26 19:27:36--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  9.45MB/s    in 0.6s    

2025-01-26 19:27:37 (9.45 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: data/ml-1m/
  inflating: data/ml-1m/movies.dat   
  inflating: data/ml-1m/ratings.dat  
  inflating: data/ml-1m/README       
  inflating: data/ml-1m/users.dat    


In [113]:
ratings = pd.read_csv("data/ml-1m/ratings.csv")
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")
ratings.head()

,user,item,values,timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [2]:
# Скачиваем архив с данными MovieLens 100k
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip -O ml-100k.zip

# Распаковываем архив в папку data
!unzip -uo ml-100k.zip -d data

# Приводим файл с рейтингами к формату CSV
!awk 'BEGIN { print "user,item,values,timestamp" } { gsub("\\t", ",", $0); print $0 }' data/ml-100k/u.data > data/ml-100k/ratings.csv

# Приводим файл с фильмами к формату CSV
!awk 'BEGIN { print "item,title,genres" } { gsub("\\|", ",", $0); print $0 }' data/ml-100k/u.item > data/ml-100k/movies.csv


--2025-01-26 18:26:43--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  7.88MB/s    in 0.6s    

2025-01-26 18:26:44 (7.88 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: data/ml-100k/
  inflating: data/ml-100k/allbut.pl  
  inflating: data/ml-100k/mku.sh     
  inflating: data/ml-100k/README     
  inflating: data/ml-100k/u.data     
  inflating: data/ml-100k/u.genre    
  inflating: data/ml-100k/u.info     
  inflating: data/ml-100k/u.item     
  inflating: data/ml-100k/u.occupation  
  inflating: data/ml-100k/u.user     
  inflating: data/ml-100k/u1.base    
  inflating: data/ml-100k/u1.test    
  inflating: data/

In [3]:
ratings = pd.read_csv("data/ml-100k/ratings.csv")
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"], unit="s")
ratings.head()

,user,item,values,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


Так как функция потерь $L$ выпукла вниз, то необходимо и достаточно, чтобы $\large \frac{\partial L}{\partial q_i} = \frac{\partial L}{\partial p_u} = 0$. Тогда, перепишем в матричном виде:

Предположим, что $Q$ - матрица констант.
$$
\frac{\partial L}{\partial p_u} = (p^T_u Q^t-r_u)\centerdot Q +\lambda p^T_u= 0
$$$$
\Rightarrow p_u^T(Q^TQ+\lambda I)=r_u \centerdot Q
$$
Тогда, $$
p_u^T=r_u \centerdot Q(Q^TQ + \lambda I)^{-1}
$$
Следовательно,
$$
p_u^T=r_u \centerdot Q(Q^TQ + \lambda I)^{-1}
$$$$
q_i^T=r_i \centerdot P(P^TP + \lambda I)^{-1}
$$

_____________________________________________________________________________


In [114]:
import numpy as np
from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.linalg import solve

class iALSRecommender:
    def __init__(self, R, num_factors=10, lamb=0.01, iterations=10):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.num_factors = num_factors
        self.lamb = lamb
        self.iterations = iterations


        self.P = np.random.rand(self.num_users, self.num_factors)
        self.Q = np.random.rand(self.num_items, self.num_factors)

    def fit(self):
        """Fit the model using ALS."""
        for iteration in range(self.iterations):
            print(f"Iteration {iteration + 1}/{self.iterations}")

            # Фиксируем Q и решае.
            QTQ = self.Q.T.dot(self.Q)
            lambdaI = eye(self.num_factors) * self.lamb

            for u in range(self.P.shape[0]):
                    self.P[u, :] = solve((QTQ + lambdaI),
                                    (self.R[u, :] @ (self.Q)).flatten())
            # Фиксируем P
            PTP = self.P.T.dot(self.P)
            lambdaI = eye(self.num_factors) * self.lamb

            PTP = self.P.T.dot(self.P)
            lambdaI = scs.eye(PTP.shape[0]) * self.lamb
            for i in range(self.Q.shape[0]):
                    self.Q[i, :] = solve((PTP + lambdaI),
                                    (self.R[:, i].T @ (self.P)).flatten())

    def predict(self):
        """Make predictions by computing the dot product of P and Q."""
        return self.P.dot(self.Q.T)

    def predict_user(self, uid):
        """Make predictions by computing the dot product of P and Q."""
        pred_ratings = self.P[uid] @ self.Q.T
        return np.argsort(pred_ratings)[::-1]




In [179]:
def train_test_split(ratings):
    train_data = []
    test_data = []

    for user, group in ratings.groupby('user'):
        group = group.sort_values('timestamp')
        train_data.append(group.iloc[:-1])
        test_data.append(group.iloc[-1])


    train_data = pd.concat(train_data)
    test_data = pd.DataFrame(test_data)

    return train_data, test_data

In [116]:
train_ratings, test_ratings = train_test_split(ratings)

In [117]:
R_train = coo_matrix((train_ratings['values'], (train_ratings['user'] - 1, train_ratings['item'] - 1))).tocsr()

In [118]:
model = iALSRecommender(R_train, num_factors=10, lamb=0.1, iterations=10)
model.fit()

predictions = model.predict()

Iteration 1/10
Iteration 2/10
Iteration 3/10
Iteration 4/10
Iteration 5/10
Iteration 6/10
Iteration 7/10
Iteration 8/10
Iteration 9/10
Iteration 10/10


In [119]:
hits = 0
for _, row in test_ratings.iterrows():
    user_id = row['user'] - 1
    true_item = row['item'] - 1

    # Предсказания для пользователя
    user_predictions = predictions[user_id, :].copy()

    # Фильтрация известных объектов
    known_items = R_train[user_id, :].nonzero()[1]
    user_predictions[known_items] = -np.inf  # Исключаем известные объекты

    # Формирование топ-10
    top_10_items = np.argsort(user_predictions)[-10:][::-1]

    #print(f"User: {user_id}, True item: {true_item}, Top-10: {top_10_items}")

    # Проверка попадания
    if true_item in top_10_items:
        hits += 1

# Расчёт метрики
hit_rate = hits / len(test_ratings)
print(f"Hit Rate: {hit_rate}")


Hit Rate: 0.07003311258278146


#SVD

In [153]:
from scipy.sparse.linalg import svds

In [ ]:
train_ratings, test_ratings = train_test_split(ratings)

In [177]:
num_users = ratings['user'].max()
num_items = ratings['item'].max()
R_train = np.zeros((num_users, num_items))

for _, row in train_ratings.iterrows():
    user_id = row['user'] - 1
    item_id = row['item'] - 1
    R_train[user_id, item_id] = row['values']

In [ ]:
U, sigma, Vt = svds(R_train, 50)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt)

In [178]:
hits = 0
for _, row in test_ratings.iterrows():
    user_id = row['user'] - 1
    true_item = row['item'] - 1


    user_predictions = predicted_ratings[user_id, :]
    known_items = np.where(R_train[user_id, :] > 0)[0]
    user_predictions[known_items] = -np.inf

    top_10_items = np.argsort(user_predictions)[-10:][::-1]

    if true_item in top_10_items:
        hits += 1

hit_rate = hits / len(test_ratings)
print(f"Hit Rate: {hit_rate:}")

Hit Rate: 0.0956953642384106


#Bandit

In [ ]:
import pandas as pd

movie_columns = ["item", "title", "release_date", "video_release_date", "IMDb_URL",
                 "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
                 "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
                 "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]


movies_raw = pd.read_csv(
    "data/ml-100k/u.item",
    sep="|",
    header=None,
    names=movie_columns,
    encoding="latin-1",
    on_bad_lines="skip"
)

movies = movies_raw[["item", "title", "unknown", "Action", "Comedy", "Drama", "Romance", "Sci-Fi"]]
movies.to_csv("data/ml-100k/movies.csv", index=False, encoding="latin-1")


In [ ]:
movies

,item,title,unknown,Action,Comedy,Drama,Romance,Sci-Fi
0,1,Toy Story (1995),0,0,1,0,0,0
1,2,GoldenEye (1995),0,1,0,0,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0
3,4,Get Shorty (1995),0,1,1,1,0,0
4,5,Copycat (1995),0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,1,0,0
1678,1679,B. Monkey (1998),0,0,0,0,1,0
1679,1680,Sliding Doors (1998),0,0,0,1,1,0
1680,1681,You So Crazy (1994),0,0,1,0,0,0


In [ ]:
def train_test_split(ratings):
    train_data = []
    test_data = []

    for user, group in ratings.groupby('user'):
        group = group.sort_values('timestamp')
        train_data.append(group.iloc[:-1])
        test_data.append(group.iloc[-1])

    train_data = pd.concat(train_data)
    test_data = pd.DataFrame(test_data)

    return train_data, test_data



In [ ]:
train_ratings, test_ratings = train_test_split(ratings)

In [ ]:
user_encoder = OneHotEncoder()
user_features = user_encoder.fit_transform(train_ratings[["user"]]).toarray()

genre_columns = ["Action", "Comedy", "Drama", "Romance", "Sci-Fi"]
movie_features = movies[genre_columns].values


movie_ids = train_ratings["item"].values - 1  # Индексы фильмов в train_ratings
movie_features_repeated = movie_features[movie_ids]


contexts = np.hstack([user_features, movie_features_repeated])
rewards = (train_ratings["values"] >= 4).astype(int).values  #+1 reward

print(f"Размеры контекста: {contexts.shape}")


Размеры контекста: (99057, 948)


In [ ]:
def LinUCB(contexts, rewards, alpha=1.0, num_arms=None):
    num_samples, num_features = contexts.shape
    A = []
    b = []
    for _ in range(num_arms):
      A.append(np.eye(num_features))
    for _ in range(num_arms):
      b.append(np.zeros(num_features))

    total_reward = 0
    predictions = []

    for t in range(0, num_samples, 1000):
        x_t = contexts[t]
        a_t = []
        #print(t)

        for arm in range(num_arms):
            theta = np.linalg.inv(A[arm]) @ b[arm]
            p =  x_t.T @ theta  + alpha * np.sqrt(x_t.T @ np.linalg.inv(A[arm]) @ x_t)
            a_t.append(p)

        chosen_arm = np.argmax(a_t)
        predictions.append(chosen_arm)

        # Обновляем параметры
        reward = rewards[t]
        A[chosen_arm] += x_t * x_t.T
        b[chosen_arm] += reward * x_t

        total_reward += reward

    return total_reward, np.array(predictions)

alpha = 1
num_arms = 2

total_reward, predictions = LinUCB(contexts, rewards, alpha, num_arms)
print(f"Reward: {total_reward}")

Reward: 58


In [ ]:
test_ratings

,user,item,values,timestamp
19699,1,102,2,889751736
7519,2,281,3,888980240
37188,3,181,4,889237482
48826,4,11,4,892004520
17458,5,457,1,879198898
...,...,...,...,...
94676,939,254,3,880262319
82942,940,316,4,889480582
98516,941,1,5,875049144
72746,942,31,5,891283517
